# 1. Import Libraries

In [ ]:
import sqlite3
import pandas as pd

# 2. Read in the Data Tables

In [23]:
# Connect to the sqlite database and read the tables into pandas dataframe objects
conn = sqlite3.connect('coffee.db')
cursor = conn.cursor()

snd_tbl_query = "SELECT * FROM SND"
coffee_type_tbl_query = "SELECT * FROM coffee_type"
market_tbl_query = "SELECT * FROM market"
price_tbl_query = "SELECT * FROM price"

df_snd = pd.read_sql_query(snd_tbl_query, conn)
df_coffee_type = pd.read_sql_query(coffee_type_tbl_query, conn)
df_market = pd.read_sql_query(market_tbl_query, conn)
df_price = pd.read_sql_query(price_tbl_query, conn)

conn.close()

In [35]:
# Convert date columns from 'object' type to 'datetime64' type
df_snd['date'] = pd.to_datetime(df_snd['date'], format='%Y-%m-%d')
df_market['date'] = pd.to_datetime(df_market['date'], format='%Y-%m-%d')
df_price['date'] = pd.to_datetime(df_price['date'], format='%Y-%m-%d')